# MeshCat Animations

MeshCat.jl also provides an animation interface, built on top of the [three.js animation system](https://threejs.org/docs/#manual/introduction/Animation-system). While it is possible to construct animation clips and tracks manually, just as you would in Three.js, it's generally easier to use the MeshCat `Animation` type.

Let's show off building a simple animation. We first have to create our scene: 

In [1]:
import meshcat
from meshcat.geometry import Box

In [2]:
vis = meshcat.Visualizer()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [3]:
## To open the visualizer in a new browser tab, do: 
# vis.open()

## To open the visualizer inside this jupyter notebook, do: 
# vis.jupyter_cell()

In [4]:
vis["box1"].set_object(Box([0.1, 0.2, 0.3]))

### Building an Animation

We construct an animation by first creating a blank `Animation()` object. We can then use the `at_frame` method to set properties or transforms of the animation at specific frames of the animation. Three.js will automatically interpolate between whatever values we provide. 

For example, let's animate moving the box from [0, 0, 0] to [0, 1, 0]: 

In [5]:
from meshcat.animation import Animation
import meshcat.transformations as tf

In [6]:
anim = Animation()

with anim.at_frame(vis, 0) as frame:
    # `frame` behaves like a Visualizer, in that we can
    # call `set_transform` and `set_property` on it, but
    # it just stores information inside the animation
    # rather than changing the current visualization
    frame["box1"].set_transform(tf.translation_matrix([0, 0, 0]))
with anim.at_frame(vis, 30) as frame:
    frame["box1"].set_transform(tf.translation_matrix([0, 1, 0]))
    
# `set_animation` actually sends the animation to the
# viewer. By default, the viewer will play the animation
# right away. To avoid that, you can also pass `play=false`. 
vis.set_animation(anim)

You should see the box slide 1 meter to the right in the viewer. If you missed the animation, you can run it again from the viewer. Click "Open Controls", find the "Animations" section, and click "play". 

### Animating the Camera

The camera is just another object in the MeshCat scene. To set its transform, we just need to index into the visualizer with the right path (note the leading `/`):

In [7]:
vis["/Cameras/default"].set_transform(tf.translation_matrix([0, 0, 1]))

To animate the camera, we just have to do that same kind of `settransform!` to individual frames in an animation: 

In [8]:
anim = Animation()

with anim.at_frame(vis, 0) as frame:
    frame["/Cameras/default"].set_transform(tf.translation_matrix([0, 0, 0]))
with anim.at_frame(vis, 30) as frame:
    frame["/Cameras/default"].set_transform(tf.translation_matrix([0, 0, 1]))

# we can repeat the animation playback with the 
# repetitions argument:
vis.set_animation(anim, repetitions=2)

We can also animate object properties. For example, let's animate the camera's `zoom` property to smoothly zoom out and then back in. Note that to do this, we have to access a deeper path in the visualizer to get to the actual camera object. For more information, see: https://github.com/rdeits/meshcat#camera-control

In [11]:
anim = Animation()

camera_path = "/Cameras/default/rotated/<object>"

with anim.at_frame(vis, 0) as frame:
    frame[camera_path].set_property("zoom", "number", 1)
with anim.at_frame(vis, 30) as frame:
    frame[camera_path].set_property("zoom", "number", 0.5)
with anim.at_frame(vis, 60) as frame:
    frame[camera_path].set_property("zoom", "number", 1)
    
# While we're animating the camera zoom, we can also animate any other
# properties we want. Let's simultaneously translate the box during 
# the same animation:
with anim.at_frame(vis, 0) as frame:
    frame["box1"].set_transform(tf.translation_matrix([0, -1, 0]))
with anim.at_frame(vis, 60) as frame:
    frame["box1"].set_transform(tf.translation_matrix([0, 1, 0]))

vis.set_animation(anim)

### Recording an Animation

To record an animation at a smooth, fixed frame rate, click on "Open Controls" in the viewer, and then go to "Animations" -> "default" -> "Recording" -> "record". This will play the entire animation, recording every frame and then let you download the resulting frames to your computer. 

To record activity in the MeshCat window that isn't a MeshCat animation, we suggest using a screen-capture tool like Quicktime for macOS or RecordMyDesktop for Linux. 

### Converting the Animation into a Video

Currently, meshcat can only save an animation as a `.tar` file consisting of a  list of `.png` images, one for each frame. To convert that into a video, you will need to install the `ffmpeg` program, and then you can run: 

In [1]:
from meshcat.animation import convert_frames_to_video

In [3]:
convert_frames_to_video("/home/rdeits/Downloads/meshcat_1528401494656.tar", overwrite=True)

Saved output as /home/rdeits/locomotion/explorations/meshcat-distro/meshcat-python/output.mp4
